# BioMed: Information Retrieval - BioMedical Information Retrieval System

---

**Group:**
- Reyes Castro, Didier Yamil (didier.reyes.castro@alumnos.upm.es)
- Rodriguez Fernández, Cristina ()

**Course:** BioMedical Informatics - 2025/26

**Institution:** Polytechnic University of Madrid (UPM)

**Date:** November 2026

---

## Goal

To develop an Information Retrieval system — specifically, a **binary text classifier** — to identify scientific articles in the PubMed database that are related to a given set of abstracts within a defined research topic. In this case, the focus is on a collection of 1,308 manuscripts containing information on the polyphenol composition of various foods.

## Setup and Installation

In [ ]:
# !pip install scikit-learn pandas requests transformers pytorch datasets numpy

In [7]:
import requests
import time

import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import Dataset
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, classification_report

## **Task 1:** 

Retrieve from PubMed the abstracts associated with each publication in publications.xlsx

In [ ]:
dataset = pd.read_csv('publications.csv')
dataset.head()

In [ ]:
BASE_URL = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/"
ESEARCH_URL = BASE_URL + "esearch.fcgi"
FETCH_URL = BASE_URL + "efetch.fcgi"

# Step 1: Search for the PMID of the article by title
def search_pmid_by_title(title, api_key=None):
    params = {
        "db": "pubmed",
        "term": f"{title}[Title]",
        "retmode": "json",
        "api_key": api_key
    }

    try:

        response = requests.get(ESEARCH_URL, params=params)
        response.raise_for_status()
        data = response.json()

        if 'esearchresult' in data and data['esearchresult']['count'] != '0':
            return data['esearchresult']['idlist'][0]
        else:
            print(f"Found {data['esearchresult']['count']} PMIDs for title: {title}. Skipping...")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error during request for title '{title}': {e}")
        return None

# Step 2: Fetch article abstract by PMID
def fetch_abstract_by_pmid(pmid, api_key=None):
    params = {
        "db": "pubmed",
        "id": pmid,
        "retmode": "text",
        "rettype": "abstract",
        "api_key": api_key
    }

    try:
        response = requests.get(FETCH_URL, params=params)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching abstract for PMID '{pmid}': {e}")
        return None

# Process each article in the dataset
relevant_abstracts = []
for i, article in dataset.iterrows():

    article_info = {
        'id': article['id'],
        'pmid': None,
        'title': article['title'],
        'abstract': None
    }

    title = article['title']
    pmid = search_pmid_by_title(title)
    
    if pmid:
        article_info['pmid'] = pmid
        abstract = fetch_abstract_by_pmid(pmid)
        article_info['abstract'] = abstract

    relevant_abstracts.append(article_info)

    # CHANGE ME TO 0.1 IF YOU HAVE AN API KEY
    print("Sleeping for 1...")
    time.sleep(1)  # Delaying 1s to respect NCBI rate limits (3 requests per second)

# Add relevant_abstracts to a new dataset
relevant_df = pd.DataFrame(relevant_abstracts)

# Save the updated dataset
relevant_df.to_csv('publications_v2.csv', index=False)

There are a lot of PMIDs whose abstract is not available :( ... Ask professor?

## **Task 2:**

Use the EUtilities tool to search for articles whose content is not relevant to this task. Size of the dataset should be the same of relevant documents.

In [ ]:
def search_irrelevant_articles(term, count, api_key=None):
    
    print(f"Fetching {count} irrelevant articles...")

    params = {
        "db": "pubmed",
        "term": term,
        "retmode": "json",
        "retmax": count,
        "api_key": api_key
    }

    try:
        response = requests.get(ESEARCH_URL, params=params)
        response.raise_for_status()
        data = response.json()

        if 'esearchresult' in data and data['esearchresult']['count'] != '0':
            return data['esearchresult']['idlist']
        else:
            print(f"Found {data['esearchresult']['count']} irrelevant articles.")
            return []

    except requests.exceptions.RequestException as e:
        print(f"Error during request for irrelevant articles: {e}")
        return []

irrelevant_pmids_list = search_irrelevant_articles("cancer[Title]", len(dataset))

irrelevant_abstracts = []
for pmid in irrelevant_pmids_list:

    article_info = {
        'pmid': pmid,
        'abstract': None
    }

    article_info['abstract'] = fetch_abstract_by_pmid(pmid)
    irrelevant_abstracts.append(article_info)

    # CHANGE ME TO 0.1 IF YOU HAVE AN API KEY
    print("Sleeping for 1...")
    time.sleep(1)  # Delaying 1s to respect NCBI rate limits (3 requests per second)

# Save irrelevant abstracts to a new dataset
irrelevant_df = pd.DataFrame(irrelevant_abstracts)
irrelevant_df.to_csv('irrelevant_publications.csv', index=False)

In [ ]:
# TODO: CHECK THIS OUT!! SEEMS NOT TO BE WORKING
# There are strange abstract in the irrelevant dataset like "1.", erasing them and researching...
# irrelevant_dataset_cleaned = irrelevant_dataset[~irrelevant_dataset['abstract'].str.match('1.')]
#
# irrelevant_dataset_cleaned

In [ ]:
# Fetching other irrelevant abstracts
# new_irrelevant_pmids_list = search_irrelevant_articles("pneumonia[Title]", len(dataset) - len(irrelevant_dataset), "8e029cc2ba291ed9ee30e494f27c18017408")
# new_irrelevant_abstracts = []
# for pmid in new_irrelevant_pmids_list:
#     abstract = fetch_abstract_by_pmid(pmid, "8e029cc2ba291ed9ee30e494f27c18017408")
#     new_irrelevant_abstracts.append(abstract)
#     print("Sleeping for 0.1...")
#     time.sleep(0.1)  # Delaying 0.1s to respect NCBI rate limits (10 requests per second)
# 
# # Adding abstracts to the irrelevant dataset
# new_irrelevant_dataset = pd.DataFrame({'pmid': new_irrelevant_pmids_list, 'abstract': new_irrelevant_abstracts})
# irrelevant_dataset = pd.concat([irrelevant_dataset, new_irrelevant_dataset], ignore_index=True)
# irrelevant_dataset.to_csv('irrelevant_publications_v2.csv', index=False)

## **Task 4:**

Implement the chosen retrieval system using the programming language of their choice. If the information retrieval system is based on machine learning techniques, the student must split the existing datasets (relevant and non-relevant documents) into three distinct groups (training, validation, and testing) to carry out the model training.

**CHOSEN RETRIEVAL SYSTEM:** BioBERT-based Binary Text Classifier

In [ ]:
# Adding target variable 'relevance' 
relevant_df['relevance'] = 1
irrelevant_df['relevance'] = 0

# Combining relevant and irrelevant datasets and maintaining only abstract and relevance columns
features = ['abstract', 'relevance']
combined_df = pd.concat([relevant_df[features], irrelevant_df[features]], ignore_index=True)

# Remove any rows where the abstract is missing (e.g., API fetch failed)
combined_df.dropna(subset=['abstract'], inplace=True)
combined_df.reset_index(drop=True, inplace=True)

# Saving
combined_df.to_csv('combined_publications.csv', index=False)

print("Class distribution:")
print(combined_df['relevance'].value_counts())

Following Fine-tuning of BERT for text classification tasks: https://huggingface.co/docs/transformers/en/tasks/sequence_classification

- Train-Test-Validation Split: 80%-10%-10%

In [ ]:
RANDOM_STATE = 42

train_df, test_df = train_test_split(combined_df,
                                     test_size=0.2,
                                     stratify=combined_df["relevance"],
                                     random_state=RANDOM_STATE)

val_df, test_df = train_test_split(test_df,
                                   test_size=0.5,
                                   stratify=test_df["relevance"],
                                   random_state=RANDOM_STATE)

print(f"Training size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df)}")

- Convert Pandas DataFrame to HuggingFace Dataset

In [ ]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

- Tokenization of abstracts using BioBERT tokenizer

In [ ]:
BERT_MODEL_NAME = "dmis-lab/biobert-v1.1"
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)

def tokenize(examples):
    return tokenizer(examples["abstract"], 
                     padding="max_length", 
                     truncation=True,
                     max_length=512 # Maximum length for BERT models
                    )

train_dataset = train_dataset.map(tokenize, batched=True)
val_dataset = val_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

# Renaming the target column to 'labels' as expected by HuggingFace Trainer
train_dataset = train_dataset.rename_column("relevance", "labels")
val_dataset = val_dataset.rename_column("relevance", "labels")
test_dataset = test_dataset.rename_column("relevance", "labels")

- Loading BioBERT model for binary text classification (relevant vs irrelevant)

In [ ]:
id2label = {0: "irrelevant", 1: "relevant"}
label2id = {"irrelevant": 0, "relevant": 1}

model = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL_NAME, 
                                                           num_labels=2,
                                                           id2label=id2label,
                                                           label2id=label2id)

- Defining evaluation metrics

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, 
                                                               predictions, 
                                                               average="binary",
                                                               zero_division=0)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

- Putting the training arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./biobert_pubmed_classifier",

    # Training hyperparameters
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,

    # Optimiser settings
    weight_decay=0.01,
    
    # Evaluation settings
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=100,

    # Model selection    
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,

    # Performance
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=4,

    seed=RANDOM_STATE,
    push_to_hub=False,
    report_to="none"
)

- Actual training using Trainer API

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

- Evaluating on the test set

In [ ]:
predictions_output = trainer.predict(test_dataset)
predictions = np.argmax(predictions_output.predictions, axis=-1)
true_labels = predictions_output.label_ids

# Calculate all metrics
test_metrics = compute_metrics((predictions_output.predictions, true_labels))

print("\nTest Set Results:")
print(f"Accuracy:  {test_metrics['accuracy']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall:    {test_metrics['recall']:.4f}")
print(f"F1-Score:  {test_metrics['f1']:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(
    true_labels, 
    predictions,
    target_names=['Irrelevant', 'Relevant'],
    digits=4
))

# Confusion matrix
print("\nConfusion Matrix:")
cm = confusion_matrix(true_labels, predictions)
print(cm)
print(f"\nTrue Negatives:  {cm[0][0]} (correctly identified irrelevant)")
print(f"False Positives: {cm[0][1]} (incorrectly marked relevant)")
print(f"False Negatives: {cm[1][0]} (missed relevant papers)")
print(f"True Positives:  {cm[1][1]} (correctly identified relevant)")

- saving the trained model

In [ ]:
model_save_path = './final_biobert_classifier'
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)